In [1]:
#Capstone IBM Data Science Project Notebook SK, Points 1-4 in the assignment

In [2]:
# We start by importing all the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [3]:
# Let us get the wikipedia link now and use beautiful soup to parse the the HTML codes

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text
makemysoup = BeautifulSoup(raw_wikipedia_page,'xml')

In [4]:
# Now we must extract the table from the page and process it clean
# Ensure that we take care the Not Assigned boroughs

mytable = makemysoup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for torontop in mytable.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for toronto_td in torontop.find_all('td'):
        if counter == 1: 
            Postcode_var = toronto_td.text
        if counter == 2: 
            Borough_var = toronto_td.text
            tag_a_Borough = toronto_td.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(toronto_td.text).strip()
            tag_a_Neighbourhood = toronto_td.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [5]:
#How many unique post codes do we have?
unique_postcode = set(Postcode)

Postcode_to      = []
Borough_to       = []
Neighbourhood_to = []


for kodikos in unique_postcode:
    posty = ''; boroy = ''; neiby = ''; 
    for postcode_part1, postcode_part2 in enumerate(Postcode):
        if kodikos == postcode_part2:
            posty = postcode_part2;
            boroy = Borough[postcode_part1]
            if neiby == '': 
                neiby = Neighbourhood[postcode_part1]
            else:
                neiby = neiby + ', ' + Neighbourhood[postcode_part1]
    Postcode_to.append(posty)
    Borough_to.append(boroy)
    Neighbourhood_to.append(neiby)

In [6]:
#Now that we found the number of unique post codes and we are happy we must put it in
#a nice dataframe and print it proper

toronto_dict = {'Postcode':Postcode_to, 'Borough':Borough_to, 'Neighbourhood':Neighbourhood_to}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto

,Borough,Neighbourhood,Postcode
0,North York,Glencairn,M6B
1,East York,Leaside,M4G
2,East Toronto,Riverdale,M4K
3,West Toronto,High Park,M6P
4,Downtown Toronto,"Harbourfront, Regent Park",M5A
5,Scarborough,"Agincourt North, Milliken",M1V
6,North York,Parkwoods,M3A
7,North York,Humber Summit,M9L
8,Scarborough,Upper Rouge,M1X
9,Downtown Toronto,"First Canadian Place, Underground city",M5X


In [7]:
#Let us use the shape method and print the rows
df_toronto.shape

(84, 3)

In [ ]:
#This is Points 1-4 from the capstone project - Part 1